# Using TensorBoard with PyTorch

- TensorBoard, as mentioned earlier, is a visualization toolkit we could use in order to further understand our model
    - It essentially reads data from a file and displays it
        - Track and visualize metrics (loss, accuracy)
        - Network graph
        - Histograms of weights and biases
- In order to use TensorBoard, we need to write the data into a file that TensorBoard can read
    - PyTorch has a utility class which we can use for this called SummaryWriter 
- Run the following
    - **tensorboard --logdir=runs**
    - The above command will write a runs file containing data written to it using the SummaryWriter() class

In [6]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms 
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# print format
torch.set_printoptions(linewidth=120) 
from torch.utils.tensorboard import SummaryWriter
#! tensorboard --version

#### Define Network

In [7]:
# Implementing the forward method
class Network(nn.Module): # extending nn.Module base class
    def __init__(self):
        super(Network, self).__init__() # initializing base class
        # prebuilt layers
        # 1 input channel, convolved by 6 different filters
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        # fully connected, or dense layers 
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden layer reshape
        # 4*4 -> height * width -> reduction due to conv operations
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer (10 classes)
        # using the softmax fucntion which returns a positive probability that sum to 1
        # but we won't use it here because it will be there in the loss function
        # which will implicitly execute the softmax function 
        t = self.out(t)
        # t = F.softmax(t, dim=1) -> done in the loss part implicitly
        
        return t

In [8]:
train_set = torchvision.datasets.FashionMNIST(
    root='./Documents/data'
     ,train=True
    ,download=True # downloads it locally (checks existence beforehand)
    ,transform=transforms.Compose([
        transforms.ToTensor() # butilt in tensor transformer
    ])
)

In [ ]:
network = Network()

batch_size = 100
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
optimizer = optim.Adam(network.parameters(), lr=0.01)

num_epochs = 5
# loop over all epochs
for epoch in range(num_epochs):
    
    # variables to track
    total_loss = 0
    total_correct = 0
    
    # loop over all batches in the train loader
    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad() # zero grad because pytorch accumulates gradient
        loss.backward() # calculate gradients
        optimizer.step() # update weights

        # update variables
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    # print information
    print("Epoch: ", epoch+1, "\n\tAccuracy (%):", total_correct/len(train_set),
          "\n\tLoss ", total_loss)


print("\nNumber of steps taken towards the loss minimum:", len(train_set)/batch_size)